In [ ]:
%load_ext autoreload
%autoreload 2

import zarr
import xarray as xr
import os
import numpy as np
import matplotlib.pyplot as plt

from carbonplan.data import cat
from cmip6_downscaling.disagg import terraclimate
from cmip6_downscaling.disagg.derived_variables import process

In [ ]:
from dask.distributed import Client

client = Client(n_workers=8, threads_per_worker=1)
client

In [ ]:
ds_conus = cat.terraclimate.terraclimate.to_dask()
ds_grid = cat.grids.conus4k.to_dask().drop(["lat", "lon", "lon_b", "lat_b"])

In [ ]:
ds_conus = process(ds_conus)
ds_conus = ds_conus.update(ds_grid)
ds_conus

In [ ]:
ds_conus["awc"] = np.maximum(ds_conus["awc"], ds_conus["soil"].max("time"))

In [ ]:
data = {}
# somewhere in idaho
data["idaho"] = (
    ds_conus.isel(x=200, y=200).squeeze(drop=True).load().to_dataframe()
)

# kitsap peninsula
data["kitsap"] = (
    ds_conus.sel(x=-2e6, y=3e6, method="nearest")
    .squeeze(drop=True)
    .load()
    .to_dataframe()
)

# west slopes colorado rockies
data["colorado"] = (
    ds_conus.sel(x=-1e6, y=2e6, method="nearest")
    .squeeze(drop=True)
    .load()
    .to_dataframe()
)

# southeast
data["southeast"] = (
    ds_conus.sel(x=1e6, y=1e6, method="nearest")
    .squeeze(drop=True)
    .load()
    .to_dataframe()
)

In [ ]:
# run our version of the terraclimate hydrology model for all points
v2 = {}
for k, df_point in data.items():
    awc = df_point["awc"][0]
    elev = df_point["elevation"][0]
    lat = df_point["lat"][0]

    v2[k] = terraclimate.model(df_point, awc, lat, elev)

In [ ]:
def plot_points(v1, v2, var="pdsi", tslice=slice(200, 300)):

    fig, axes = plt.subplots(ncols=len(v2), figsize=(24, 4))

    for i, k in enumerate(v2):
        v1[k][var][tslice].plot(label="v1", ax=axes[i])
        v2[k][var][tslice].plot(label="v2", ax=axes[i])
        axes[i].set_title(k.title())

    plt.ylabel(var)
    plt.legend()

    return fig, axes

In [ ]:
fig, axes = plot_points(data, v2, var="pdsi")

In [ ]:
fig, axes = plot_points(data, v2, var="pet")

In [ ]:
fig, axes = plot_points(data, v2, var="swe")

In [ ]:
fig, axes = plot_points(data, v2, var="aet")

In [ ]:
fig, axes = plot_points(data, v2, var="soil")

In [ ]:
fig, axes = plot_points(data, v2, var="q")

In [ ]:
fig, axes = plot_points(data, v2, var="def")